In [ ]:
import os
print(os.listdir("../input"))

In [ ]:
import gc
import logging #библиотека для удобного ведения логов в Python
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
#settings
warnings.filterwarnings('ignore')
np.random.seed(2018)

In [ ]:
#logger
def get_logger():
    FORMAT = '[%(levelname)s]%(asctime)s:%(name)s:%(message)s'
    logging.basicConfig(format=FORMAT) #в настройки логера передать уровень выводимых ошибок
    logger = logging.getLogger('main')
    logger.setLevel(logging.DEBUG) # Сообщение отладочное
    return logger

In [ ]:
logger = get_logger()
#load data
logger.info('Start read data')
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
historical_trans_df = pd.read_csv('../input/historical_transactions.csv')
new_merchant_trans_df = pd.read_csv('../input/new_merchant_transactions.csv')
merchants = pd.read_csv('../input/merchants.csv')
#merchants = pd.read_csv('../input/merchants.csv', skiprows=lambda i: skip_func(i,p=1))

In [ ]:
def print_null(df):
    for col in df:
        if df[col].isnull().any():
            print('%s has %.0f null values: %.3f%%'%(col, df[col].isnull().sum(), df[col].isnull().sum()/df[col].count()*100))

In [ ]:
def impute_na(X_train, df, variable):
    # make temporary df copy
    temp = df.copy()
    
    # extract random from train set to fill the na
    random_sample = X_train[variable].dropna().sample(temp[variable].isnull().sum(), random_state=0, replace=True)
    
    # pandas needs to have the same index in order to merge datasets
    random_sample.index = temp[temp[variable].isnull()].index
    temp.loc[temp[variable].isnull(), variable] = random_sample
    return temp[variable]

In [ ]:
# Clipping outliers
def clipping_outliers(X_train, df, var):
    IQR = X_train[var].quantile(0.75)-X_train[var].quantile(0.25)
    lower_bound = X_train[var].quantile(0.25) - 6*IQR
    upper_bound = X_train[var].quantile(0.75) + 6*IQR
    no_outliers = len(df[df[var]>upper_bound]) + len(df[df[var]<lower_bound])
    print('There are %i outliers in %s: %.3f%%' %(no_outliers, var, no_outliers/len(df)))
    df[var] = df[var].clip(lower_bound, upper_bound)
    return df

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
merchants_df = pd.read_csv('../input/merchants.csv')
merchants_df.shape

In [ ]:
merchants_df = merchants_df.replace([np.inf,-np.inf], np.nan)

In [ ]:
print_null(merchants_df)

In [ ]:
# Average sales null
null_cols = ['avg_purchases_lag3','avg_sales_lag3', 'avg_purchases_lag6','avg_sales_lag6','avg_purchases_lag12','avg_sales_lag12']
for col in null_cols:
    merchants_df[col] = merchants_df[col].fillna(merchants_df[col].mean())

# Category 2
merchants_df['category_2'] = impute_na(merchants_df, merchants_df, 'category_2')

In [ ]:
merchants_df['most_recent_sales_range'].value_counts().sort_values(ascending=False).values

In [ ]:
merchants_df['most_recent_purchases_range'].value_counts()

In [ ]:
# Sales cut Сокращение продаж
sales_cut = merchants_df['most_recent_sales_range'].value_counts().sort_values(ascending=False).values
sales_cut = sales_cut/np.sum(sales_cut)
for i in range(1,len(sales_cut)):
    sales_cut[i] = sales_cut[i]+sales_cut[i-1]
    
# Purchases cut Покупки сокращены
purchases_cut = merchants_df['most_recent_purchases_range'].value_counts().sort_values(ascending=False).values
purchases_cut = purchases_cut/np.sum(purchases_cut)
for i in range(1,len(purchases_cut)):
    purchases_cut[i] = purchases_cut[i]+purchases_cut[i-1]

In [ ]:
sales_cut, purchases_cut

In [ ]:
merchants_df.shape

In [ ]:
merchants_df.head()

In [ ]:
# Discretize data Дискретизировать данные
discretize_cols = ['avg_purchases_lag3','avg_sales_lag3', 'avg_purchases_lag6','avg_sales_lag6','avg_purchases_lag12','avg_sales_lag12']

for col in discretize_cols:
    categories = pd.qcut(merchants_df[col].values,sales_cut, duplicates='raise').categories.format()
    merchants_df[col], intervals = pd.qcut(merchants_df[col], 5, labels=['A','B','C','D','E'], retbins=True, duplicates='raise')
    print('Discretize for %s:'%col)
    print(categories)

In [ ]:
merchants_df.head()

In [ ]:
# Mapping data
merchants_df['category_1'] = merchants_df['category_1'].map({'Y':1, 'N':0})
merchants_df['category_4'] = merchants_df['category_4'].map({'Y':1, 'N':0})

map_cols = discretize_cols + ['most_recent_purchases_range', 'most_recent_sales_range']
for col in map_cols:
    merchants_df[col] = merchants_df[col].map({'A':5,'B':4,'C':3,'D':2,'E':1})

In [ ]:
map_cols

In [ ]:
merchants_df.head()

In [ ]:
numeric_cols = ['numerical_1','numerical_2']+map_cols

colormap = plt.cm.RdBu
plt.figure(figsize=(12,12))
sns.heatmap(merchants_df[numeric_cols].astype(float).corr(), linewidths=0.1, vmax=1.0, vmin=-1., square=True, cmap=colormap, linecolor='white', annot=True)
plt.title('Pair-wise correlation')

In [ ]:
#Обработка числовых данных
numerical_cols = ['numerical_1','numerical_2']
for col in numerical_cols:
    merchants_df = clipping_outliers(merchants_df, merchants_df, col)
    plt.figure()
    sns.distplot(merchants_df[col])
print('Unique values:')
print(merchants_df[numerical_cols].nunique())

In [ ]:
#После отсечения выбросов в этих двух столбцах осталось только 5 уникальных значений. 
#Таким образом, мы отображаем их на 3 категории: самая низкая: 0, средняя: 1 и крайняя: 2
for col in numerical_cols:
    b = merchants_df[col].unique()
    merchants_df[col] = merchants_df[col].apply(lambda x: 0 if x==b[0] else (1 if x in b[1:4] else 2))                

In [ ]:
merchants_df = reduce_mem_usage(merchants_df)

In [ ]:
# Rename col
for col in merchants_df.columns:
    if col != 'merchant_id':
        merchants_df = merchants_df.rename(index=str, columns={col:'mer_'+col})

In [ ]:
merchants_df.head()

min_amount = historical_trans_df.purchase_amount.min()
historical_trans_df.purchase_amount = historical_trans_df.purchase_amount.apply(lambda x: x-min_amount+1)
min_amount = new_merchant_trans_df.purchase_amount.min()
new_merchant_trans_df.purchase_amount = new_merchant_trans_df.purchase_amount.apply(lambda x: x-min_amount+1)

In [ ]:
#process NAs
logger.info('Start processing NAs')
#process NA2 for transactions
for df in [historical_trans_df, new_merchant_trans_df]:
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True) #historical_trans_df.merchant_id.describe().top
#define function for aggregation
def create_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [ ]:
# Merge with merchant data
historical_trans_df = pd.merge(historical_trans_df, merchants_df, how='left', left_on='merchant_id', right_on='merchant_id')
#new_historical_trans_df = pd.merge(new_historical_trans_df, merchants_df, how='left', left_on='merchant_id', right_on='merchant_id')

del merchants_df
gc.collect()

In [ ]:
historical_trans_df = reduce_mem_usage(historical_trans_df)
historical_trans_df = historical_trans_df.drop(columns=['mer_city_id', 'mer_state_id', 'mer_category_1', 'mer_category_2',
                          'mer_merchant_category_id','mer_subsector_id'])
gc.collect()

In [ ]:
logger.info('process historical and new merchant datasets')
for df in [historical_trans_df, new_merchant_trans_df]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    df['category_3'] = df['category_3'].map({'A':0, 'B':1, 'C':2}) 
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30 #Кол-во дней с покупки
    df['month_diff'] += df['month_lag']
    df['is_month_start'] = df['purchase_date'].dt.is_month_start.apply(lambda x: 1 if x == 'True' else 0)#map({'False':0, 'True':1})
    df['is_month_end'] = df['purchase_date'].dt.is_month_end.apply(lambda x: 1 if x == 'True' else 0)#.map({'False':0, 'True':1})
    #Черная пятница
    df['weeks_to_BFriday'] = ((pd.to_datetime('2017-11-25') - df['purchase_date']).dt.days//7).apply(lambda x: x if x>=0 and x<=3 else 3)
    #Рождество
    df['weeks_to_Xmas_2017'] = ((pd.to_datetime('2017-12-25') - df['purchase_date']).dt.days//7).apply(lambda x: x if x>=0 and x<=8 else 8)
    

In [ ]:
historical_trans_df.head()

In [ ]:
#define aggregations with historical_trans_df
logger.info('Aggregate historical trans')
aggs = {}

for col in ['subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
for col in ['month', 'hour', 'weekofyear', 'dayofweek', 'year']:
    aggs[col] = ['nunique', 'mean', 'min', 'max']
    
aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min','count']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean', 'min', 'max', 'var']
aggs['authorized_flag'] = ['sum', 'mean', 'min', 'max']
aggs['weekend'] = ['sum', 'mean', 'min', 'max']
aggs['category_1'] = ['sum', 'mean', 'min']
aggs['category_2'] = ['sum', 'mean', 'min']
aggs['category_3'] = ['sum', 'mean', 'min']
aggs['card_id'] = ['size', 'count']
aggs['weeks_to_BFriday'] = ['sum', 'mean', 'min', 'max']
aggs['weeks_to_Xmas_2017'] = ['sum', 'mean', 'min', 'max']
aggs['is_month_start'] = ['sum', 'mean', 'min', 'max']
aggs['is_month_end'] = ['sum', 'mean', 'min', 'max']

for col in historical_trans_df.columns:
    if 'mer_' in col:
        #print (i)
        aggs[col] = ['nunique', 'mean', 'min', 'max', 'count']

for col in ['category_2','category_3']:
    historical_trans_df[col+'_mean'] = historical_trans_df.groupby([col])['purchase_amount'].transform('mean')
    historical_trans_df[col+'_min'] = historical_trans_df.groupby([col])['purchase_amount'].transform('min')
    historical_trans_df[col+'_max'] = historical_trans_df.groupby([col])['purchase_amount'].transform('max')
    historical_trans_df[col+'_sum'] = historical_trans_df.groupby([col])['purchase_amount'].transform('sum')
    aggs[col+'_mean'] = ['mean']    

new_columns = create_new_columns('hist',aggs)
historical_trans_group_df = historical_trans_df.groupby('card_id').agg(aggs)
historical_trans_group_df.columns = new_columns
historical_trans_group_df.reset_index(drop=False,inplace=True)
historical_trans_group_df['hist_purchase_date_diff'] = (historical_trans_group_df['hist_purchase_date_max'] - historical_trans_group_df['hist_purchase_date_min']).dt.days
historical_trans_group_df['hist_purchase_date_average'] = historical_trans_group_df['hist_purchase_date_diff']/historical_trans_group_df['hist_card_id_size']
historical_trans_group_df['hist_purchase_date_uptonow'] = (datetime.datetime.today() - historical_trans_group_df['hist_purchase_date_max']).dt.days
historical_trans_group_df['hist_purchase_date_uptomin'] = (datetime.datetime.today() - historical_trans_group_df['hist_purchase_date_min']).dt.days
#merge with train, test
train_df = train_df.merge(historical_trans_group_df,on='card_id',how='left')
test_df = test_df.merge(historical_trans_group_df,on='card_id',how='left')


In [ ]:
#cleanup memory
del historical_trans_group_df; 
gc.collect()

In [ ]:
#define aggregations with new_merchant_trans_df 
logger.info('Aggregate new merchant trans')
aggs = {}
for col in ['subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
for col in ['month', 'hour', 'weekofyear', 'dayofweek', 'year']:
    aggs[col] = ['nunique', 'mean', 'min', 'max']

    
aggs['purchase_amount'] = ['sum','max','min','mean','var','count']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min', 'count']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean', 'max', 'min', 'var']
aggs['weekend'] = ['sum', 'mean', 'min']
aggs['category_1'] = ['sum', 'mean', 'min']
aggs['category_2'] = ['sum', 'mean', 'min']
aggs['category_3'] = ['sum', 'mean', 'min']
aggs['card_id'] = ['size', 'count']
aggs['is_month_start'] = ['sum', 'mean', 'min', 'max']
aggs['is_month_end'] = ['sum', 'mean', 'min', 'max']
aggs['weeks_to_BFriday'] = ['sum', 'mean', 'min', 'max']
aggs['weeks_to_Xmas_2017'] = ['sum', 'mean', 'min', 'max']

for col in ['category_2','category_3']:
    new_merchant_trans_df[col+'_mean'] = new_merchant_trans_df.groupby([col])['purchase_amount'].transform('mean')
    new_merchant_trans_df[col+'_min'] = new_merchant_trans_df.groupby([col])['purchase_amount'].transform('min')
    new_merchant_trans_df[col+'_max'] = new_merchant_trans_df.groupby([col])['purchase_amount'].transform('max')
    new_merchant_trans_df[col+'_sum'] = new_merchant_trans_df.groupby([col])['purchase_amount'].transform('sum')
    aggs[col+'_mean'] = ['mean']


    
new_columns = create_new_columns('new_hist',aggs)
new_merchant_trans_group_df = new_merchant_trans_df.groupby('card_id').agg(aggs)
new_merchant_trans_group_df.columns = new_columns
new_merchant_trans_group_df.reset_index(drop=False,inplace=True)
new_merchant_trans_group_df['new_hist_purchase_date_diff'] = (new_merchant_trans_group_df['new_hist_purchase_date_max'] - new_merchant_trans_group_df['new_hist_purchase_date_min']).dt.days
new_merchant_trans_group_df['new_hist_purchase_date_average'] = new_merchant_trans_group_df['new_hist_purchase_date_diff']/new_merchant_trans_group_df['new_hist_card_id_size']
new_merchant_trans_group_df['new_hist_purchase_date_uptonow'] = (datetime.datetime.today() - new_merchant_trans_group_df['new_hist_purchase_date_max']).dt.days
new_merchant_trans_group_df['new_hist_purchase_date_uptomin'] = (datetime.datetime.today() - new_merchant_trans_group_df['new_hist_purchase_date_min']).dt.days
#merge with train, test
train_df = train_df.merge(new_merchant_trans_group_df,on='card_id',how='left')
test_df = test_df.merge(new_merchant_trans_group_df,on='card_id',how='left')


In [ ]:
#clean-up memory
del new_merchant_trans_group_df; gc.collect()
del historical_trans_df; gc.collect()
del new_merchant_trans_df; gc.collect()

In [ ]:
#process train
logger.info('Process train')
train_df['outliers'] = 0
train_df.loc[train_df['target'] < -30, 'outliers'] = 1
train_df['outliers'].value_counts()
logger.info('Process train and test')


In [ ]:
## process both train and test
for df in [train_df, test_df]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['dayofyear'] = df['first_active_month'].dt.dayofyear
    df['quarter'] = df['first_active_month'].dt.quarter
    df['is_month_start'] = df['first_active_month'].dt.is_month_start
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['hist_last_buy'] = (df['hist_purchase_date_max'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_last_buy'] = (df['new_hist_purchase_date_max'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['card_id_cnt_total'] = df['new_hist_card_id_count']+df['hist_card_id_count']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']
    df['purchase_amount_mean'] = df['new_hist_purchase_amount_mean']+df['hist_purchase_amount_mean']
    df['purchase_amount_max'] = df['new_hist_purchase_amount_max']+df['hist_purchase_amount_max']

for f in ['feature_1','feature_2','feature_3']:
    order_label = train_df.groupby([f])['outliers'].mean()
    train_df[f] = train_df[f].map(order_label)
    test_df[f] = test_df[f].map(order_label)
##
#train_columns = [c for c in train_df.columns if c not in ['card_id', 'first_active_month','target','outliers']]
#target = train_df['target']
#del train_df['target']


In [ ]:
train_df.to_csv("train_ds1.csv", index=False)
test_df.to_csv("test_ds1.csv", index=False)